# Standard GP

In [1]:
import fastgps
import qmcpy as qp
import torch
import numpy as np

In [2]:
device = "cpu"
if device!="mps":
    torch.set_default_dtype(torch.float64)

## True Function

In [3]:
def f_ackley(x, a=20, b=0.2, c=2*np.pi, scaling=32.768):
    # https://www.sfu.ca/~ssurjano/ackley.html
    assert x.ndim==2
    x = 2*scaling*x-scaling
    t1 = a*torch.exp(-b*torch.sqrt(torch.mean(x**2,1)))
    t2 = torch.exp(torch.mean(torch.cos(c*x),1))
    t3 = a+np.exp(1)
    y = -t1-t2+t3
    return y
f_low_fidelity = lambda x: f_ackley(x,c=0)
f_high_fidelity = lambda x: f_ackley(x)
f_cos = lambda x: torch.cos(2*np.pi*x).sum(1)
fs = [f_low_fidelity,f_high_fidelity,f_cos]
d = 1 # dimension
rng = torch.Generator().manual_seed(17)
x = torch.rand((2**7,d),generator=rng).to(device) # random testing locations
y = torch.vstack([f(x) for f in fs]) # true values at random testing locations
z = torch.rand((2**8,d),generator=rng).to(device) # other random locations at which to evaluate covariance
print("x.shape = %s"%str(tuple(x.shape)))
print("y.shape = %s"%str(tuple(y.shape)))
print("z.shape = %s"%str(tuple(z.shape)))

x.shape = (128, 1)
y.shape = (3, 128)
z.shape = (256, 1)


## Construct Fast GP

In [4]:
fgp = fastgps.StandardGP(
    kernel = qp.KernelMultiTask(
        qp.KernelSquaredExponential(d,torchify=True,device=device),
        num_tasks=len(fs)),
    seqs = 7)
x_next = fgp.get_x_next(n=[2**4,2**2,2**3])
y_next = [fs[i](x_next[i]) for i in range(fgp.num_tasks)]
fgp.add_y_next(y_next)
assert len(x_next)==len(y_next)
for i in range(len(x_next)):
    print("i = %d"%i)
    print("\tx_next[%d].shape = %s"%(i,str(tuple(x_next[i].shape))))
    print("\ty_next[%d].shape = %s"%(i,str(tuple(y_next[i].shape))))

i = 0
	x_next[0].shape = (16, 1)
	y_next[0].shape = (16,)
i = 1
	x_next[1].shape = (4, 1)
	y_next[1].shape = (4,)
i = 2
	x_next[2].shape = (8, 1)
	y_next[2].shape = (8,)


In [5]:
pmean = fgp.post_mean(x)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("l2 relative error =",(torch.linalg.norm(y-pmean,dim=1)/torch.linalg.norm(y,dim=1)))

pmean.shape = (3, 128)
l2 relative error = tensor([0.1861, 0.2679, 0.0975])


In [6]:
data = fgp.fit()
list(data.keys())

     iter of 5.0e+03 | best loss  | loss      
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            0.00e+00 | 6.91e+05   | 6.91e+05  
            5.00e+00 | 2.97e+05   | 2.97e+05  
            1.00e+01 | 8.64e+01   | 8.64e+01  
            1.50e+01 | 7.87e+01   | 8.03e+01  
            2.00e+01 | 6.65e+01   | 6.65e+01  
            2.50e+01 | 5.07e+01   | 5.07e+01  
            3.00e+01 | 4.96e+01   | 4.96e+01  
            3.50e+01 | 4.87e+01   | 4.87e+01  
            4.00e+01 | 4.76e+01   | 4.76e+01  
            4.50e+01 | 4.73e+01   | 4.73e+01  
            5.00e+01 | 4.73e+01   | 4.73e+01  
            5.30e+01 | 4.73e+01   | 4.73e+01  


[]

In [7]:
pmean,pvar,q,ci_low,ci_high = fgp.post_ci(x,confidence=0.99)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("pvar.shape = %s"%str(tuple(pvar.shape)))
print("q = %.2f"%q)
print("ci_low.shape = %s"%str(tuple(ci_low.shape)))
print("ci_high.shape = %s"%str(tuple(ci_high.shape)))
print("l2 relative error =",(torch.linalg.norm(y-pmean,dim=1)/torch.linalg.norm(y,dim=1)))
pcov = fgp.post_cov(x,x)
print("pcov.shape = %s"%str(tuple(pcov.shape)))
_range0,_rangen1 = torch.arange(pcov.size(0)),torch.arange(pcov.size(-1))
pcov2 = fgp.post_cov(x,z)
print("pcov2.shape = %s"%str(tuple(pcov2.shape)))
print("\npcov diag matches pvar: %s"%torch.allclose(pcov[_range0,_range0][:,_rangen1,_rangen1],pvar))
print("non-negative pvar: %s"%(pvar>=0).all().item())

pmean.shape = (3, 128)
pvar.shape = (3, 128)
q = 2.58
ci_low.shape = (3, 128)
ci_high.shape = (3, 128)
l2 relative error = tensor([0.0315, 0.1709, 0.2248])
pcov.shape = (3, 3, 128, 128)
pcov2.shape = (3, 3, 128, 256)

pcov diag matches pvar: True
non-negative pvar: True


In [8]:
pcmean,pcvar,q,cci_low,cci_high = fgp.post_cubature_ci(confidence=0.99)
print("pcmean =",pcmean)
print("pcvar =",pcvar)
print("cci_low =",cci_low)
print("cci_high",cci_high)

pcmean = tensor([ 1.7030e+01,  1.9753e+01, -5.4638e-03])
pcvar = tensor([0.0007, 0.0464, 0.0016])
cci_low = tensor([16.9613, 19.1977, -0.1090])
cci_high tensor([17.0985, 20.3074,  0.0981])


## Project and Increase Sample Size

In [9]:
n_new = fgp.n*torch.tensor([4,2,8],device=device)
pcov_future = fgp.post_cov(x,z,n=n_new)
pvar_future = fgp.post_var(x,n=n_new)
pcvar_future = fgp.post_cubature_var(n=n_new)

In [10]:
x_next = fgp.get_x_next(n_new)
y_next = [fs[i](x_next[i]) for i in range(fgp.num_tasks)]
for _y in y_next:
    print(_y.shape)
fgp.add_y_next(y_next)
print("l2 relative error =",(torch.linalg.norm(y-fgp.post_mean(x),dim=1)/torch.linalg.norm(y,dim=1)))
assert torch.allclose(fgp.post_cov(x,z),pcov_future)
assert torch.allclose(fgp.post_var(x),pvar_future)
assert torch.allclose(fgp.post_cubature_var(),pcvar_future)

torch.Size([48])
torch.Size([4])
torch.Size([56])
l2 relative error = tensor([0.0243, 0.1789, 0.2022])


In [11]:
data = fgp.fit(verbose=False)
print("l2 relative error =",(torch.linalg.norm(y-fgp.post_mean(x),dim=1)/torch.linalg.norm(y,dim=1)))

l2 relative error = tensor([0.0565, 0.1815, 0.3245])


In [12]:
n_new = fgp.n*torch.tensor([4,8,2],device=device)
pcov_new = fgp.post_cov(x,z,n=n_new)
pvar_new = fgp.post_var(x,n=n_new)
pcvar_new = fgp.post_cubature_var(n=n_new)
x_next = fgp.get_x_next(n_new)
y_next = [fs[i](x_next[i]) for i in range(fgp.num_tasks)]
fgp.add_y_next(y_next)
assert torch.allclose(fgp.post_cov(x,z),pcov_new)
assert torch.allclose(fgp.post_var(x),pvar_new)
assert torch.allclose(fgp.post_cubature_var(),pcvar_new)